In [9]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

# ================== PARAMETRI ==================
# Path al CSV di input
INTERACTIONS_PATH = r"J:/Cohen Lab/Maria Clara/2_Lab data/9_Napari/OUTPUT new code/iN day28 D14mito D7 perox/neurites/01012026_iNday28_neurites_interactions.csv"

# Path all'Excel di output
OUTPUT_EXCEL = r"J:/Cohen Lab/Maria Clara/2_Lab data/9_Napari/k-way NEW 022026/k-way_iNday28_neurites_022026.xlsx"
# ==============================================

def object_k_from_name(obj: str) -> int:
    """Inferisci il k-way dal nome oggetto: numero di 'X' + 1."""
    if not isinstance(obj, str):
        return np.nan
    return obj.count("X") + 1

def compute_cell_fraction(cell_k_wide: pd.DataFrame) -> pd.DataFrame:
    """
    Calcola, per ogni k-way e per ogni filtro (all, f0, f1),
    quante cellule hanno >=1 contatto (vs 0 contatti) e la relativa %.
    Usa le colonne wide del tipo: n_all_k2, n_f0_k3, n_f1_k4, ecc.
    """
    if "image_name" not in cell_k_wide.columns:
        raise ValueError("cell_k_wide deve contenere la colonna 'image_name'.")

    total_cells = cell_k_wide["image_name"].nunique()
    rows = []

    prefixes = [("all", "n_all_"), ("f0", "n_f0_"), ("f1", "n_f1_")]

    for filt, pref in prefixes:
        k_cols = [c for c in cell_k_wide.columns if c.startswith(pref) and c.split(pref, 1)[1].startswith("k")]
        # ordina per k numerico
        def k_num(col):
            m = re.search(r"_k(\d+)$", col)
            return int(m.group(1)) if m else 10**9
        k_cols = sorted(k_cols, key=k_num)

        for col in k_cols:
            m = re.search(r"_k(\d+)$", col)
            if not m:
                continue
            k = int(m.group(1))

            s = pd.to_numeric(cell_k_wide[col], errors="coerce").fillna(0)
            n_ge1 = int((s >= 1).sum())
            n_zero = int((s == 0).sum())
            # Nota: se ci sono NaN originali, vengono conteggiati come 0 (coerente con fillna(0) usato nel pipeline)

            rows.append({
                "filter": filt,
                "k": k,
                "cells_total": int(total_cells),
                "cells_with_>=1": n_ge1,
                "cells_with_0": n_zero,
                "pct_cells_with_>=1": (n_ge1 / total_cells * 100) if total_cells else np.nan,
                "pct_cells_with_0": (n_zero / total_cells * 100) if total_cells else np.nan,
            })

    out = pd.DataFrame(rows)
    if not out.empty:
        out = out.sort_values(["filter", "k"]).reset_index(drop=True)
    return out

def build_obj_summaries(interactions_path: str, output_path: str):
    # Carica interactions
    df = pd.read_csv(interactions_path)

    # k_from_object = 2-way, 3-way, ecc (numero di 'X' + 1)
    df["k_from_object"] = df["object"].astype(str).apply(object_k_from_name)

    group_keys = ["image_name", "object"]

    # ---- TUTTI i contatti (no filtro in_higher_order) ----
    overall = (
        df.groupby(group_keys, as_index=False)
          .agg(
              total_count_all=("label", "size"),
              volume_sum_all=("volume", "sum"),
              volume_mean_all=("volume", "mean"),
              volume_median_all=("volume", "median"),
              volume_std_all=("volume", "std"),
          )
    )

    # Aggiungi k_from_object
    k_map = df[group_keys + ["k_from_object"]].drop_duplicates()
    overall = overall.merge(k_map, on=group_keys, how="left")

    # ---- in_higher_order = 0 ----
    f0 = (
        df[df["in_higher_order"] == 0]
          .groupby(group_keys, as_index=False)
          .agg(
              total_count_f0=("label", "size"),
              volume_sum_f0=("volume", "sum"),
              volume_mean_f0=("volume", "mean"),
              volume_median_f0=("volume", "median"),
              volume_std_f0=("volume", "std"),
          )
    )

    # ---- in_higher_order = 1 ----
    f1 = (
        df[df["in_higher_order"] == 1]
          .groupby(group_keys, as_index=False)
          .agg(
              total_count_f1=("label", "size"),
              volume_sum_f1=("volume", "sum"),
              volume_mean_f1=("volume", "mean"),
              volume_median_f1=("volume", "median"),
              volume_std_f1=("volume", "std"),
          )
    )

    # Merge per oggetto
    obj_summary = (
        overall
        .merge(f0, on=group_keys, how="left")
        .merge(f1, on=group_keys, how="left")
    )

    # Metti a 0 i NaN sulle metriche
    cols_to_fill = [
        c for c in obj_summary.columns
        if c not in group_keys + ["k_from_object"]
    ]
    obj_summary[cols_to_fill] = obj_summary[cols_to_fill].fillna(0)

    # ---- Per-cellula: totali ----
    cell_totals = (
        obj_summary
        .groupby("image_name", as_index=False)
        .agg(
            total_contacts_all=("total_count_all", "sum"),
            total_volume_all=("volume_sum_all", "sum"),
            total_contacts_f0=("total_count_f0", "sum"),
            total_volume_f0=("volume_sum_f0", "sum"),
            total_contacts_f1=("total_count_f1", "sum"),
            total_volume_f1=("volume_sum_f1", "sum"),
        )
    )

    # ---- Per-cellula × k-way (2-,3-,4-way...) ----
    cell_k = (
        obj_summary
        .groupby(["image_name", "k_from_object"], as_index=False)
        .agg(
            total_contacts_all=("total_count_all", "sum"),
            total_volume_all=("volume_sum_all", "sum"),
            total_contacts_f0=("total_count_f0", "sum"),
            total_volume_f0=("volume_sum_f0", "sum"),
            total_contacts_f1=("total_count_f1", "sum"),
            total_volume_f1=("volume_sum_f1", "sum"),
        )
    )

    # ---- Versione wide per GraphPad ----
    def pivot_k_metric(dfk, metric_col, prefix):
        p = dfk.pivot(index="image_name", columns="k_from_object", values=metric_col)
        p.columns = [f"{prefix}_k{int(c)}" for c in p.columns]
        return p

    metrics_for_k = [
        ("total_contacts_all", "n_all"),
        ("total_volume_all", "vol_all"),
        ("total_contacts_f0", "n_f0"),
        ("total_volume_f0", "vol_f0"),
        ("total_contacts_f1", "n_f1"),
        ("total_volume_f1", "vol_f1"),
    ]

    wide_parts = []
    for col, prefix in metrics_for_k:
        wide_parts.append(pivot_k_metric(cell_k, col, prefix))

    cell_k_wide = pd.concat(wide_parts, axis=1).reset_index()

    # ===== NUOVO OUTPUT: frazione di cellule con >=1 contatto per k-way =====
    cell_fraction = compute_cell_fraction(cell_k_wide)

    # ---- Salva Excel ----
    out_path = Path(output_path)
    out_path.parent.mkdir(parents=True, exist_ok=True)

    with pd.ExcelWriter(out_path) as writer:
        obj_summary.to_excel(writer, sheet_name="by_object", index=False)
        cell_totals.to_excel(writer, sheet_name="by_cell_totals", index=False)
        cell_k.to_excel(writer, sheet_name="by_cell_kway_long", index=False)
        cell_k_wide.to_excel(writer, sheet_name="by_cell_kway_wide", index=False)
        cell_fraction.to_excel(writer, sheet_name="cell_fraction", index=False)

    print(f"Salvato: {out_path}")

# Esegui
build_obj_summaries(INTERACTIONS_PATH, OUTPUT_EXCEL)


Salvato: J:\Cohen Lab\Maria Clara\2_Lab data\9_Napari\k-way NEW 022026\k-way_iNday28_neurites_022026.xlsx
